Импортируем библиотеки

In [36]:
import requests
from bs4 import BeautifulSoup as BS
import unidecode
import pandas as pd
import re
from datetime import date
import locale
locale.setlocale(locale.LC_ALL, 'ru_RU.UTF-8')
import pymorphy2

Задаем ссылку с Drom.ru для парсинга (можно из активных объявлений или из архива)

In [22]:
url = r'https://novokuznetsk.drom.ru/volkswagen/passat/generation5/restyling0/page8/?neighborhood=1&transmission[]=1&fueltype=1&damaged=2'

Парсим объявления (максимум 400, дром больше не даёт)

In [23]:
#создаем пустой df
df = pd.DataFrame()

#чек следующей страницы
next_page = True
#начинаем с первой страницы
n = 1

while next_page == True:
    #если пользователь дал ссылку, где уже есть page - меняем page по порядку
    if url.find('page') >=0:
        url_1 = re.sub(r'page.*/', fr'page{n}/', url)        
    #если ссылка без page - добавляем page по порядку
    else:
        url_1 = url.replace(r'/?', fr'/page{n}/?')
    print(f'Страница №{n}. Ссылка: {url_1}')
    #получаем страницу в xml
    r = requests.get(url_1) #verify = False, если ошибка SSL
    #беру код страницы сверху вниз вплоть до слов "Объявления в архиве/в ближайших городах"
    #если не находим, то будет -1 и последний символ страницы не попадёт, но в рамках задачи это не важно
    r = r.text[:r.text.find('Объявления в ')]
    #читаем супом
    soup = BS(r, 'lxml')
    
    #для каждого блока с объявлением
    for i in soup.findAll('a', class_='css-xb5nz8 e1huvdhj1'):

        #некоторых элементов может не быть
        try:
            engine = i.findAll('span', class_='css-1l9tp44 e162wx9x0')[0].text
        except:
            engine = None
        try:
            fuel = i.findAll('span', class_='css-1l9tp44 e162wx9x0')[1].text
        except:
            fuel = None
        try:
            transmission = i.findAll('span', class_='css-1l9tp44 e162wx9x0')[2].text
        except:
            transmission = None
        try:
            wd = i.findAll('span', class_='css-1l9tp44 e162wx9x0')[3].text
        except:
            wd = None
            
        #записывем в df2 найденное в объявлении
        df2 = pd.DataFrame({'date':[i.find('div', attrs={'data-ftid': 'bull_date'}).text],
                            'name':[i.find('span').text],
                            'engine':[engine],
                            'fuel':[fuel],
                            'transmission':[transmission],
                            'wd':[wd],
                            'price':[unidecode.unidecode(i.find('span', class_='css-46itwz e162wx9x0').text).replace(' ','').replace('R', '')],
                            'city':[i.find('span', class_='css-1488ad e162wx9x0').text],
                             #вместо try-except
                            'active':[getattr(i.find('div',class_='css-z5srlr e162wx9x0'), 'text', 'активно')], 
                            'link':[i.get('href')]})
        #добавляем инфу по объявлению в общий df
        df = pd.concat([df, df2])
    #пробуем найти ссылку на следующую страницу:
    #если нашли - увеличиваем счетчик страниц n
    try:
        soup.find('a', class_='css-4gbnjj e24vrp30').get('href')
        n = n + 1
    #если не нашли - чек следующей страницы False, завершаем while
    except:
        next_page = False
        print('Последняя страница')
        
#обновляем индексы (сейчас они у нас все == 0)        
df = df.reset_index()
df.drop(columns = 'index', inplace = True)

#если у даты объявления нет даты, то текущая дата
df['date'] = df['date'].apply(lambda x: f"{date.today().strftime('%d %B %Y')}" if re.match('.*час.*', str(x)) else x)
#если у даты объявления нет года, значит год текущий
df['date'] = df['date'].apply(lambda x: x+f" {date.today().year}" if not re.match('.*20\d{2}', str(x)) else x)


# инициализируем парсер для разбора слов
m = pymorphy2.MorphAnalyzer()   
#вытаскиваем месяц в отдельный столбец
df['month'] = df.date.str.split(' ', expand = True)[1].fillna('без_месяца')
#применяем pymorphy2 для перевода в именительный падеж
df['month'] = df['month'].apply(lambda x: m.parse(x)[0].inflect({'nomn'}).word.title())
#заменяем месяц в оригинальном столбце месяцем  в именительном падеже 
df['date'] = df.apply(lambda x: re.sub(r'\s\w+\s', f' {x.month} ', x['date']), axis=1)

df.drop(columns = 'month', inplace = True)
#переводим в нужный тип
df['price'] = df['price'].astype('int')
#когда дата уже в норм типе - NaN и заполняем их из ещё существующего date
df['date'] = pd.to_datetime(df['date'], format='%d %B %Y', errors = 'coerce')
#убираем надоедливые запятые во всей таблице
df.replace(',', '', regex= True, inplace = True)

Страница №1. Ссылка: https://novokuznetsk.drom.ru/volkswagen/passat/generation5/restyling0/page1/?neighborhood=1&transmission[]=1&fueltype=1&damaged=2
Страница №2. Ссылка: https://novokuznetsk.drom.ru/volkswagen/passat/generation5/restyling0/page2/?neighborhood=1&transmission[]=1&fueltype=1&damaged=2
Страница №3. Ссылка: https://novokuznetsk.drom.ru/volkswagen/passat/generation5/restyling0/page3/?neighborhood=1&transmission[]=1&fueltype=1&damaged=2
Страница №4. Ссылка: https://novokuznetsk.drom.ru/volkswagen/passat/generation5/restyling0/page4/?neighborhood=1&transmission[]=1&fueltype=1&damaged=2
Страница №5. Ссылка: https://novokuznetsk.drom.ru/volkswagen/passat/generation5/restyling0/page5/?neighborhood=1&transmission[]=1&fueltype=1&damaged=2
Страница №6. Ссылка: https://novokuznetsk.drom.ru/volkswagen/passat/generation5/restyling0/page6/?neighborhood=1&transmission[]=1&fueltype=1&damaged=2
Страница №7. Ссылка: https://novokuznetsk.drom.ru/volkswagen/passat/generation5/restyling0/pag

Полученные данные

In [28]:
df

,date,name,engine,fuel,transmission,wd,price,city,active,link
0,2023-03-18,Volkswagen Passat 1998,1.6 л (100 л.с.),бензин,механика,передний,140000,Прокопьевск (≈ 40 км),активно,https://prokopyevsk.drom.ru/volkswagen/passat/...
1,2023-03-07,Volkswagen Passat 1999,1.6 л (100 л.с.),бензин,механика,передний,285000,Инской (≈ 110 км),снят с продажи,https://inskoy.drom.ru/volkswagen/passat/50064...
2,2023-03-09,Volkswagen Passat 1997,1.6 л (100 л.с.),бензин,механика,передний,200000,Белово (≈ 110 км),снят с продажи,https://belovo.drom.ru/volkswagen/passat/49762...
3,2023-02-28,Volkswagen Passat 1998,1.8 л (125 л.с.),бензин,механика,передний,200000,Ленинск-Кузнецкий (≈ 140 км),активно,https://leninsk-kuznetskiy.drom.ru/volkswagen/...
4,2023-02-21,Volkswagen Passat 1999,1.6 л (100 л.с.),бензин,механика,передний,280000,Ленинск-Кузнецкий (≈ 140 км),снят с продажи,https://leninsk-kuznetskiy.drom.ru/volkswagen/...
...,...,...,...,...,...,...,...,...,...,...
156,2023-02-21,Volkswagen Passat 1998,1.8 л (125 л.с.),бензин,механика,передний,360000,Евпатория (≈ 4 800 км),снят с продажи,https://yevpatoriya.drom.ru/volkswagen/passat/...
157,2023-03-18,Volkswagen Passat 1998,1.8 л (125 л.с.),бензин,механика,передний,250000,Севастополь (≈ 4 900 км),активно,https://sevastopol.drom.ru/volkswagen/passat/5...
158,2023-02-27,Volkswagen Passat 1998,1.8 л (125 л.с.),бензин,механика,передний,150000,Севастополь (≈ 4 900 км),снят с продажи,https://sevastopol.drom.ru/volkswagen/passat/4...
159,2023-02-25,Volkswagen Passat 2000,1.8 л (125 л.с.),бензин,механика,передний,350000,Севастополь (≈ 4 900 км),снят с продажи,https://sevastopol.drom.ru/volkswagen/passat/4...
